In [1]:
import pandas as pd
df = pd.read_excel('All_MHB_ES_Pval.xlsx')


In [2]:
# Step 2: Define lowest 2 and highest 3 concentration categorization
df['Concentration_Rank'] = df.groupby(['Well_Category', 'Drug'])['Well_concentration'].rank(method='dense', ascending=True)
df['Lowest_2'] = df['Concentration_Rank'] <= 2  # True for lowest 2 concentrations
df['Highest_3'] = df['Concentration_Rank'] > 2  # True for highest 3 concentrations
filtered_df = df[df['Well_Category'] == 'BET']

filtered_df_low = filtered_df[filtered_df['Lowest_2'] == True]


In [3]:
# Step 3: Group and aggregate data for counts
result = df.groupby('Well_Category').apply(lambda group: pd.Series({
    'Lowest_2_Conc_Count': group['Lowest_2'].sum(),
    'Highest_3_Conc_Count': group['Highest_3'].sum(),
    'Lowest_2_Conc_Enriched_Count': group[group['Lowest_2'] & (group['Pvalue'] < 0.05)].shape[0],
    'Highest_3_Conc_Enriched_Count': group[group['Highest_3'] & (group['Pvalue'] < 0.05)].shape[0],
})).reset_index()


/tmp/ipykernel_1633102/3137974455.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('Well_Category').apply(lambda group: pd.Series({


In [4]:
# Step 4: Calculate enrichment percentages
result['Lowest_2_Conc_Enrichment_Percentage'] = (
    (result['Lowest_2_Conc_Enriched_Count'] / result['Lowest_2_Conc_Count']) * 100
)
result['Highest_3_Conc_Enrichment_Percentage'] = (
    (result['Highest_3_Conc_Enriched_Count'] / result['Highest_3_Conc_Count']) * 100
)
result

,Well_Category,Lowest_2_Conc_Count,Highest_3_Conc_Count,Lowest_2_Conc_Enriched_Count,Highest_3_Conc_Enriched_Count,Lowest_2_Conc_Enrichment_Percentage,Highest_3_Conc_Enrichment_Percentage
0,BET,18,27,9,23,50.000000,85.185185
1,CDK,24,36,4,17,16.666667,47.222222
2,EGFR,32,48,22,46,68.750000,95.833333
3,HDAC,42,62,24,29,57.142857,46.774194
4,MEK1/2,12,18,11,18,91.666667,100.000000
5,Mitotic,18,27,7,23,38.888889,85.185185
6,PARP,10,15,6,11,60.000000,73.333333
7,PI3K,52,77,20,15,38.461538,19.480519
8,Topoisomerase,24,36,17,18,70.833333,50.000000
9,VEGFR,38,57,21,11,55.263158,19.298246
